# MNIST Training using PyTorch

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

## Install SageMaker Python SDK with TensorFlow 2.x Support (>1.49)

In [ ]:
!pip install sagemaker --upgrade --ignore-installed --no-cache

In [ ]:
!pip list

## Create the SageMaker Session

In [35]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

## Setup the Service Execution Role and Region

In [36]:
role = get_execution_role()
print('RoleARN:  {}\n'.format(role))
region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

RoleARN:  arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-20191006T135881

Region:  us-east-1


Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [37]:
sagemaker_execution_role = sagemaker.get_execution_role()

## Training Data

In [50]:
local_data_path = './pytorch-mnist/data'

In [52]:
from torchvision import datasets, transforms

datasets.MNIST(local_data_path, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

5.7%

100.1%

Extracting ./pytorch-mnist/data/MNIST/raw/train-images-idx3-ubyte.gz to ./pytorch-mnist/data/MNIST/raw


31.3%%

Extracting ./pytorch-mnist/data/MNIST/raw/train-labels-idx1-ubyte.gz to ./pytorch-mnist/data/MNIST/raw


180.4%

Extracting ./pytorch-mnist/data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./pytorch-mnist/data/MNIST/raw
Extracting ./pytorch-mnist/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./pytorch-mnist/data/MNIST/raw
Processing...
Done!


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./pytorch-mnist/data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

There are four ``.npy`` file under this prefix:
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [53]:
!ls -R {local_data_path}

./pytorch-mnist/data:
MNIST

./pytorch-mnist/data/MNIST:
processed  raw

./pytorch-mnist/data/MNIST/processed:
test.pt  training.pt

./pytorch-mnist/data/MNIST/raw:
t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [ ]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/pytorch-mnist/data'

In [40]:
inputs = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-835319576252/sagemaker/pytorch-mnist


In [41]:
!aws s3 ls --recursive s3://{bucket}/{data_prefix}

2019-12-26 17:40:20    7920421 sagemaker/pytorch-mnist/MNIST/processed/test.pt
2019-12-26 17:40:14   47520423 sagemaker/pytorch-mnist/MNIST/processed/training.pt
2019-12-26 17:40:13    7840016 sagemaker/pytorch-mnist/MNIST/raw/t10k-images-idx3-ubyte
2019-12-26 17:40:13    1648877 sagemaker/pytorch-mnist/MNIST/raw/t10k-images-idx3-ubyte.gz
2019-12-26 17:40:13      10008 sagemaker/pytorch-mnist/MNIST/raw/t10k-labels-idx1-ubyte
2019-12-26 17:40:13       4542 sagemaker/pytorch-mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
2019-12-26 17:40:14   47040016 sagemaker/pytorch-mnist/MNIST/raw/train-images-idx3-ubyte
2019-12-26 17:40:13    9912422 sagemaker/pytorch-mnist/MNIST/raw/train-images-idx3-ubyte.gz
2019-12-26 17:40:13      60008 sagemaker/pytorch-mnist/MNIST/raw/train-labels-idx1-ubyte
2019-12-26 17:40:13      28881 sagemaker/pytorch-mnist/MNIST/raw/train-labels-idx1-ubyte.gz


## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [42]:
!ls ./src/mnist_pytorch.py

./src/mnist_pytorch.py


### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [43]:
from sagemaker.pytorch import PyTorch

mnist_estimator = PyTorch(entry_point='mnist_pytorch.py',
                          source_dir='./src',
                          role=sagemaker_execution_role,
                          framework_version='1.3.1',
                          train_instance_count=2,
                          train_instance_type='ml.p3.2xlarge',
                          hyperparameters={
                            'epochs': 6,
                            'backend': 'gloo'
                          })

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [44]:
mnist_estimator.fit(inputs={'training': inputs}, wait=False)
training_job_name = mnist_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

training_job_name:  pytorch-training-2019-12-26-17-40-26-186


In [46]:
mnist_estimator = PyTorch.attach(training_job_name=training_job_name)

2019-12-26 17:40:28 Starting - Launching requested ML instances.........
2019-12-26 17:41:34 Starting - Preparing the instances for training......
2019-12-26 17:42:41 Downloading - Downloading input data...
2019-12-26 17:43:05 Training - Downloading the training image......
2019-12-26 17:44:34 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-12-26 17:44:34,616 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-12-26 17:44:34,641 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-12-26 17:44:34,713 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-12-26 17:44:34,737 sagemaker_pytorch_container.training INFO     Block until

INFO:__main__:Train Epoch: 1 [6400/30000 (21%)] Loss: 2.110914
Distributed training - True
INFO:__main__:Train Epoch: 1 [6400/30000 (21%)] Loss: 2.066044
Distributed training - True
INFO:__main__:Train Epoch: 1 [12800/30000 (43%)] Loss: 1.214439
Number of gpus available - 1
INFO:__main__:Train Epoch: 1 [12800/30000 (43%)] Loss: 0.982412
Number of gpus available - 1
INFO:__main__:Train Epoch: 1 [19200/30000 (64%)] Loss: 0.908709
Initialized the distributed environment: 'gloo' backend on 2 nodes. Current host rank is 0. Number of gpus: 1
INFO:__main__:Train Epoch: 1 [19200/30000 (64%)] Loss: 0.964408
Initialized the distributed environment: 'gloo' backend on 2 nodes. Current host rank is 1. Number of gpus: 1
INFO:__main__:Train Epoch: 1 [25600/30000 (85%)] Loss: 0.697935
Get train data loader
INFO:__main__:Train Epoch: 1 [25600/30000 (85%)] Loss: 0.623431
Get train data loader
/opt/conda/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will


2019-12-26 17:46:03 Uploading - Uploading generated training model
2019-12-26 17:46:03 Completed - Training job completed
INFO:__main__:Test set: Average loss: 0.0938, Accuracy: 9701/10000 (97%)
Train Epoch: 5 [12800/30000 (43%)] Loss: 0.169939

Train Epoch: 5 [19200/30000 (64%)] Loss: 0.418289
INFO:__main__:Saving the model.
Train Epoch: 5 [25600/30000 (85%)] Loss: 0.272007
Test set: Average loss: 0.1042, Accuracy: 9682/10000 (97%)

Train Epoch: 6 [6400/30000 (21%)] Loss: 0.192137
Train Epoch: 6 [12800/30000 (43%)] Loss: 0.290625
Train Epoch: 6 [19200/30000 (64%)] Loss: 0.392888
Train Epoch: 6 [25600/30000 (85%)] Loss: 0.237660
Test set: Average loss: 0.0938, Accuracy: 9701/10000 (97%)

Saving the model.
[2019-12-26 17:45:53.078 algo-1:60 INFO utils.py:27] The end of training job file will not be written for jobs running under SageMaker.
2019-12-26 17:45:53,447 sagemaker-containers INFO     Reporting training SUCCESS
Training seconds: 404
Billable seconds: 404


# Option 1:  Perform Batch Predictions Directly in the Notebook

In [48]:
# TODO

# Option 2:  Perform Batch Predictions with a Batch Transform Job

In [47]:
# TODO

# Option 3:  Create a SageMaker Endpoint and Perform REST-based Predictions

## Host
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c5.2xlarge')

### Evaluate
We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which we can then pass to the `predictor`.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
print(data)

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
estimator.delete_endpoint()